# 기본적인 세팅

1. gemma3:1b 허깅페이스에서 라이선스 동의를 받아놓아야 함

2. 허깅페이스 api key를 아래에 입력해야 함

3. corpus.json을 업로드 해놓아야 함

4. A100

In [1]:
!pip install trl huggingface_hub loguru -q
!huggingface-cli login

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import json
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments
)
from trl import SFTTrainer, SFTConfig
from loguru import logger
import shutil
import sys
import os
import itertools
import platform

logger.remove()
logger.add(
    sys.stdout,
    level="INFO",
    colorize=True,
    format="<green>{time:HH:mm:ss}</green> | <level>{level: <5}</level> | {message}"
)

# 버전 로깅
logger.info(f"python version       : {platform.python_version()}")
logger.info(f"torch version        : {torch.__version__}")
logger.info(f"transformers version : {__import__('transformers').__version__}")
logger.info(f"datasets version     : {__import__('datasets').__version__}")
logger.info(f"trl version          : {__import__('trl').__version__}")

# 데이터 로드
def load_raw_data(path="/content/corpus.json"):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

# 데이터 전처리
def make_sft_data(raw_data):
    result = []
    for item in raw_data:
        instruction = item['instruction']
        keywords = item['input']
        prompt = f"{instruction}: {', '.join(keywords)}"
        chosen = item['chosen']
        result.append({
            'input': prompt,
            'target': chosen
        })
    return result

# 전처리 함수
def preprocess(example):
    input_enc = tokenizer(example["input"], truncation=True, max_length=192)
    target_enc = tokenizer(example["target"], truncation=True, max_length=192)

    return {
        "input_ids": input_enc["input_ids"],
        "attention_mask": input_enc["attention_mask"],
        "labels": target_enc["input_ids"]
    }

def test(prompt, model, tokenizer, necessary_word):
    model = model.to("cuda")

    # 토크나이즈
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # 생성
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )

    # 디코딩
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    logger.info("="*100)
    logger.info(necessary_word + "\n" + prompt)
    logger.info(necessary_word + "\n" + generated_text)
    logger.info("="*100)
    return generated_text

13:22:38 | INFO  | python version       : 3.11.12
13:22:38 | INFO  | torch version        : 2.6.0+cu124
13:22:38 | INFO  | transformers version : 4.51.3
13:22:38 | INFO  | datasets version     : 3.6.0
13:22:38 | INFO  | trl version          : 0.17.0


In [3]:
# 훈련 파라미터
num_epochs = 5
batch_size = 8
save_total_limit = 2
output_dir = "./outputs/sft"
logging_dir = "./outputs/logs"
sft_dir = "./outputs/best_sft"

# 모델 및 토크나이저 설정
model_name = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [4]:
# 문학적 instruction 목록
instructions = [
    "제공된 단어로 문학적인 어조로 짧은 장면을 창작해주세요.",
    "아래 단어들을 활용해 상징과 감정이 담긴 문학적 단편을 작성해주세요.",
    "다음 키워드를 사용해 감성적이고 은유적인 이야기를 작성해주세요.",
    "아래 키워드를 사용해 비유와 상징이 녹아든 문학적 장면을 묘사해주세요."
]

# 키워드 목록
keywords_list = [
    ["밥", "숟가락", "그릇"],
    ["창문", "바람", "햇살"],
    ["우산", "비", "골목"],
    ["신발", "거리", "그림자"],
    ["책상", "연필", "종이"],
    ["시계", "벽", "침묵"],
    ["의자", "창가", "오후"],
    ["커피", "잔", "향기"],
    ["손", "온기", "기억"],
    ["길", "노을", "발자국"]
]

necessary_word = "[Before Train]"

# 데카르트 곱을 이용해 모든 instruction-keywords 조합 생성
test_prompts = [
    f"{instruction} {', '.join(keywords)}"
    for instruction, keywords in itertools.product(instructions, keywords_list)
]

before_response = []
for prompt in test_prompts:
    before_response.append(test(prompt, model, tokenizer, necessary_word))

13:23:42 | INFO  | ====================================================================================================
13:23:42 | INFO  | [Before Train]
제공된 단어로 문학적인 어조로 짧은 장면을 창작해주세요. 밥, 숟가락, 그릇
13:23:42 | INFO  | [Before Train]
제공된 단어로 문학적인 어조로 짧은 장면을 창작해주세요. 밥, 숟가락, 그릇, 식사, 바다, 소화, 신비, 아름다움

### 제어: 단어의 의미는 단순히 단어만으로도 adequately 표현되며, 대시 문법이 포함되지 않습니다. 단어의 의미는 단순히 단어만으로도 adequately 표현되며, 대시 문법이 포함되지 않습니다.

1.  **식사** - "밥은 신비로 한 번에 다소 소화하고, 바다에서 고기와 같은 식물에서 인체에 전달된다. 소화는 신체의 수용기에서 이자적인 물질을 수용하는 데 도움을 줍니다."* * "balloons"는 기차의 대형 공기기구로, "cricket"는 종종 바다에 주로 found happens. "mermaid"는 바다에서 주로 found happening." * * "piano"는 대형 주 Instrument로, "painting"는 대형 예술의 장면을 보입니다. "table"는 대형 식기로, "book"는 대형 사서의 장면을 보입니다. "chair"는 대형 구석 구
13:23:42 | INFO  | ====================================================================================================
13:23:47 | INFO  | ====================================================================================================
13:23:47 | INFO  | [Bef

In [5]:
# pad_token 설정 및 로그
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.resize_token_embeddings(len(tokenizer))
    logger.info(f"[Tokenizer] pad_token was None. Set to eos_token: {tokenizer.pad_token}")
else:
    logger.info(f"[Tokenizer] pad_token already set: {tokenizer.pad_token}")

# 데이터 준비 및 분할
raw_data = load_raw_data()
sft_records = make_sft_data(raw_data)
dataset = Dataset.from_list(sft_records)
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset["train"].map(preprocess, remove_columns=["input", "target"])
eval_dataset = split_dataset["test"].map(preprocess, remove_columns=["input", "target"])


logger.info(f"[Raw Data] First sample:\n{json.dumps(raw_data[0], ensure_ascii=False, indent=2)}")
logger.info(f"[SFT Records] First record:\n{json.dumps(sft_records[0], ensure_ascii=False, indent=2)}")
logger.info(f"[Dataset] Total samples: {len(dataset)}")
logger.info(f"[Split] Train size: {len(split_dataset['train'])}, Eval size: {len(split_dataset['test'])}")
logger.info(f"[Train Preprocessed] Sample keys: {list(train_dataset[0].keys())}")
logger.info(f"[Train Preprocessed] input_ids length: {len(train_dataset[0]['input_ids'])}, labels length: {len(train_dataset[0]['labels'])}")


13:26:57 | INFO  | [Tokenizer] pad_token was None. Set to eos_token: <|eot_id|>


Map:   0%|          | 0/140 [00:00<?, ? examples/s]

Map:   0%|          | 0/35 [00:00<?, ? examples/s]

13:26:58 | INFO  | [Raw Data] First sample:
{
  "instruction": "아래 세 단어를 사용해 감성적인 문학적 문장을 작성해주세요.",
  "input": [
    "김치",
    "냄새",
    "반찬"
  ],
  "rejected": "김치는 대표적인 발효 음식으로 한국 식탁에 빠지지 않는 반찬입니다. 특유의 냄새는 발효 과정에서 생성되는 유산균과 향신료에서 비롯됩니다. 김치는 보관 온도와 숙성 기간에 따라 맛이 달라집니다. 대부분의 한국 가정에서는 김치 냉장고를 통해 별도 보관합니다. 다양한 요리에 활용되며 건강에도 긍정적인 영향을 줍니다.",
  "chosen": "김치는 잊힌 계절처럼 식탁 위에 조용히 놓여 있었다. 냄새는 오래된 대화처럼 천천히 방 안을 채워나갔다. 반찬 중에서도 유독 진한 그 존재감은 입보다 기억을 먼저 자극했다. 그녀는 숟가락을 들기 전, 잠시 그 냄새에 고개를 숙였다. 어떤 향기는 맛보다 먼저 마음을 물들인다."
}
13:26:58 | INFO  | [SFT Records] First record:
{
  "input": "아래 세 단어를 사용해 감성적인 문학적 문장을 작성해주세요.: 김치, 냄새, 반찬",
  "target": "김치는 잊힌 계절처럼 식탁 위에 조용히 놓여 있었다. 냄새는 오래된 대화처럼 천천히 방 안을 채워나갔다. 반찬 중에서도 유독 진한 그 존재감은 입보다 기억을 먼저 자극했다. 그녀는 숟가락을 들기 전, 잠시 그 냄새에 고개를 숙였다. 어떤 향기는 맛보다 먼저 마음을 물들인다."
}
13:26:58 | INFO  | [Dataset] Total samples: 175
13:26:58 | INFO  | [Split] Train size: 140, Eval size: 35
13:26:58 | INFO  | [Train Preprocessed] Sample keys: ['input_ids', 'attention_mask', 'labels']
13:26:58 | 

In [6]:
# 하이퍼파라미터 정의

total_train_steps = len(train_dataset) // batch_size * num_epochs
logging_steps = max(1, total_train_steps // (num_epochs * 2))
save_steps = logging_steps

# 로깅 출력
logger.info(f"Total samples (train): {len(train_dataset)}")
logger.info(f"Total samples (eval): {len(eval_dataset)}")
logger.info(f"Batch size: {batch_size}")
logger.info(f"Epochs: {num_epochs}")
logger.info(f"Total training steps: {total_train_steps}")
logger.info(f"Logging steps: {logging_steps}")
logger.info(f"Save steps: {save_steps}")

13:26:58 | INFO  | Total samples (train): 140
13:26:58 | INFO  | Total samples (eval): 35
13:26:58 | INFO  | Batch size: 8
13:26:58 | INFO  | Epochs: 5
13:26:58 | INFO  | Total training steps: 85
13:26:58 | INFO  | Logging steps: 8
13:26:58 | INFO  | Save steps: 8


In [7]:
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=batch_size,
    num_train_epochs=num_epochs,
    fp16=True,
    logging_strategy="steps",
    logging_steps=logging_steps,
    save_strategy="steps",
    save_steps=save_steps,
    eval_strategy="steps",
    eval_steps=logging_steps,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_total_limit=save_total_limit,
    report_to="none",
    logging_dir=logging_dir
)

# Trainer 설정
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# 학습 시작
trainer.train()

# 모델 저장
trainer.model.save_pretrained(sft_dir)
tokenizer.save_pretrained(sft_dir)
shutil.make_archive("best_sft", 'zip', sft_dir)

Truncating train dataset:   0%|          | 0/140 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/35 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss
8,4.073900,3.187438
16,2.666300,2.259158
24,1.790600,2.113613
32,1.478300,2.132476
40,1.343400,2.187975
48,1.026200,2.236814
56,0.890300,2.270638
64,0.587400,2.369711
72,0.650200,2.342306
80,0.388500,2.442565


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


'/content/best_sft.zip'

In [8]:
total_size = sum(os.path.getsize(os.path.join(root, f))
                 for root, _, files in os.walk(sft_dir)
                 for f in files)

logger.info(f"Total size of best_sft: {total_size / (1024**2):.2f} MB")

14:05:36 | INFO  | Total size of best_sft: 4730.74 MB


In [9]:
necessary_word = "[After Train]"

after_response = []
for prompt in test_prompts:
    after_response.append(test(prompt, model, tokenizer, necessary_word))

14:05:43 | INFO  | ====================================================================================================
14:05:43 | INFO  | [After Train]
제공된 단어로 문학적인 어조로 짧은 장면을 창작해주세요. 밥, 숟가락, 그릇
14:05:43 | INFO  | [After Train]
제공된 단어로 문학적인 어조로 짧은 장면을 창작해주세요. 밥, 숟가락, 그릇, 침묵, 목소리, 주머니, 주머니, 바지, 바지, 커피인사, 소문, 손가락, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지, 바지,
14:05:43 | INFO  | ====================================================================================================
14:05:50 | INFO  | ====================================================================================================
14:05:50 | INFO  | [After Train]
제공된 단어로 문학적인 어조로 짧은 장면을 창작해주세요. 창문, 바람, 햇살
14:05:50 | INFO  | [After Train]
제공된 단어로 문학적인 어조로 짧은 장면을 창작해주세요. 창문, 바람, 